<a href="https://colab.research.google.com/github/abhinavsathi/sms_text_classification/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y tensorflow protobuf tensorflow-datasets
!pip install tensorflow tensorflow-datasets

In [ ]:
!pip install tensorflow-datasets

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the data
train_data = pd.read_csv("train-data.tsv", sep="\t", names=["class", "text"])
test_data = pd.read_csv("valid-data.tsv", sep="\t", names=["class", "text"])

# Convert labels to numeric data (0 for ham, 1 for spam)
train_labels = train_data['class'].apply(lambda x: 0 if x == 'ham' else 1).values
test_labels = test_data['class'].apply(lambda x: 0 if x == 'ham' else 1).values

# Tokenize and pad sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_data['text'])
max_len = 150

train_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(train_data['text']), maxlen=max_len, padding='post', truncating='post')
test_padded = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(test_data['text']), maxlen=max_len, padding='post', truncating='post')




In [ ]:
# Build and compile the model
model = keras.Sequential([
    keras.layers.Embedding(len(tokenizer.word_index) + 1, 16, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_padded, train_labels, epochs=25, verbose=0)

In [ ]:

def predict_message(pred_text):

    sequence = tokenizer.texts_to_sequences([pred_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_len, padding='post', truncating='post')

    prediction_score = model.predict(padded_sequence)[0][0]

    if prediction_score > 0.5:
        label = "spam"
    else:
        label = "ham"

    return [1 - prediction_score, label]

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
